In [1]:
%load_ext autoreload
%autoreload 2

## Import Dependencies

In [2]:

import comet_ml
import sys
sys.path.append("../modules")

from pathlib import Path
from training.api import ChatAPI, FinQATrainingAPI

/home/pauliusztin/.cache/pypoetry/virtualenvs/training-6xkSxa8Q-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No logging configuration file found at: logging.yaml. Setting logging level to INFO.
INFO:training:Initializing resources...


## Train Model

In [4]:
training_api = FinQATrainingAPI(debug=True)

INFO:training.api.FinQATrainingAPI:Loading FinQA datasets from self._root_dataset_dir=PosixPath('../dataset')


Map: 100%|██████████| 1147/1147 [00:00<00:00, 9217.24 examples/s]
INFO:training.api.FinQATrainingAPI:Debug mode enabled. Truncating datasets...
INFO:training.api.FinQATrainingAPI:Loading model using self._model_id='tiiuae/falcon-7b-instruct'
Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.32s/it]


In [5]:
trainer = training_api.train()

/home/pauliusztin/.cache/pypoetry/virtualenvs/training-6xkSxa8Q-py3.11/lib/python3.11/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/iusztinpaul/hands-on-llms/dba66f39eefe461495778da20108220b

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Perplexity
3,2.327300,2.154120,0.000000
6,2.114800,2.132880,0.000000
9,1.895200,2.099824,0.000000
12,2.156300,2.060307,0.000000


INFO:accelerate.accelerator:The used dataset had no length, returning gathered tensors. You should drop the remainder yourself.
INFO:accelerate.accelerator:The used dataset had no length, returning gathered tensors. You should drop the remainder yourself.
INFO:accelerate.accelerator:The used dataset had no length, returning gathered tensors. You should drop the remainder yourself.
INFO:accelerate.accelerator:The used dataset had no length, returning gathered tensors. You should drop the remainder yourself.
INFO:accelerate.accelerator:The used dataset had no length, returning gathered tensors. You should drop the remainder yourself.
INFO:accelerate.accelerator:The used dataset had no length, returning gathered tensors. You should drop the remainder yourself.
INFO:accelerate.accelerator:The used dataset had no length, returning gathered tensors. You should drop the remainder yourself.
INFO:accelerate.accelerator:The used dataset had no length, returning gathered tensors. You should drop 

In [ ]:
trainer.state.log_history

[{'loss': 1.0479, 'learning_rate': 0.0002, 'epoch': 1.0, 'step': 1},
 {'loss': 1.0542, 'learning_rate': 0.0002, 'epoch': 2.0, 'step': 2},
 {'loss': 1.0474, 'learning_rate': 0.0002, 'epoch': 3.0, 'step': 3},
 {'eval_loss': 2.1510212421417236,
  'eval_perplexity': 6.727502466219448e-08,
  'eval_runtime': 5.4704,
  'eval_samples_per_second': 1.097,
  'eval_steps_per_second': 0.366,
  'epoch': 3.0,
  'step': 3},
 {'loss': 1.0546, 'learning_rate': 0.0002, 'epoch': 4.0, 'step': 4},
 {'loss': 1.0475, 'learning_rate': 0.0002, 'epoch': 5.0, 'step': 5},
 {'loss': 1.0368, 'learning_rate': 0.0002, 'epoch': 6.0, 'step': 6},
 {'eval_loss': 2.1235926151275635,
  'eval_perplexity': 6.37254586877134e-08,
  'eval_runtime': 5.4891,
  'eval_samples_per_second': 1.093,
  'eval_steps_per_second': 0.364,
  'epoch': 6.0,
  'step': 6},
 {'train_runtime': 135.3107,
  'train_samples_per_second': 0.399,
  'train_steps_per_second': 0.044,
  'total_flos': 982691063267328.0,
  'train_loss': 1.0480629007021587,
  'ep

## Test Model

In [ ]:
# test_dataset = finqa.FinQADataset(
#     data_path=ROOT_DATASET_DIR / "private_test.json", scope=constants.Scope.TESTING
# ).to_huggingface()
# test_dataset

In [ ]:
# test_dataset["text"][0]

In [ ]:
# chat_api = ChatAPI(peft_model_id=Path("..") / "results" / "checkpoint-280", max_new_tokens=100)

In [ ]:
# print(
#     chat_api.ask(
#         question=test_dataset["text"][0],
#     )
# )